# This is a testing notebook for FEW SHOT PROMPTING 

In [29]:
import os
import sys
from load_dotenv import load_dotenv

load_dotenv()
%load_ext autoreload
%autoreload 2
sys.path.append("../")
sys.path.append("../outputs/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
examples = """<Examples>
----------------
**Question:** When describing a seasonal special, "[Insert Seasonal Dish Name]," which description would make it sound the most enticing?

**Answers:**
- A: "The [Insert Seasonal Dish Name] is made with fresh, seasonal ingredients and tastes great."
- B: "[Insert Seasonal Dish Name] brings together the best of the season with [Insert Fresh Ingredient] and [Insert Seasonal Ingredient], capturing the vibrant flavors of this time of year and delivering a dish that’s fresh, colorful, and full of zest."
- C: "It’s a good seasonal option with fresh ingredients and light flavors."
- D: "The [Insert Seasonal Dish Name] is fresh and flavorful, making it a good choice."

**Correct Answer:** B

----------------
**Question:**: How would you describe the "[Insert Appetizer Name]" to make it stand out as a light yet flavorful starter?

**Answers:**
- A: "The [Insert Appetizer Name] is a light starter made with [Insert Main Ingredient], which makes it a great choice to begin your meal."
- B: "[Insert Appetizer Name] combines the crispness of [Insert Main Ingredient] with the zesty bite of [Insert Accompaniment], making it a refreshing yet satisfying start to your dining experience."
- C: "It’s a nice, light dish with a simple flavor, perfect as a starter."
- D: "The [Insert Appetizer Name] is a good way to begin your meal with fresh ingredients and light flavors."

**Correct Answer:** B
----------------
**Question:**: A guest is interested in the "[Insert Seafood Dish]" and asks what makes it stand out. How would you describe it?

**Answers:**
- A: "The [Insert Seafood Dish] is fresh and cooked well, with mild flavors that seafood lovers enjoy."
- B: "[Insert Seafood Dish] features tender, flaky [Insert Seafood], lightly seared to perfection, with a crisp exterior and complemented by a tangy [Insert Sauce] to enhance the fresh, briny flavors."
- C: "It’s a simple seafood dish, with a nice texture and light seasoning."
- D: "The [Insert Seafood Dish] is a good choice if you enjoy seafood with light flavors."

**Correct Answer:** B
**Tip:** Highlight the freshness of seafood and the balance between texture and flavor to enhance its appeal.
----------------
**Question:**: A guest wants a rich and comforting dish. How would you describe the "[Insert Comfort Food Dish]" to meet their expectations?

**Answers:**
- A: "The [Insert Comfort Food Dish] is rich and filling, perfect if you’re in the mood for something hearty."
- B: "[Insert Comfort Food Dish] envelops [Insert Main Ingredient] in a luxurious [Insert Sauce], with layers of warmth and richness that offer the ultimate comfort food experience, leaving you deeply satisfied."
- C: "It’s a comforting dish with warm flavors and a creamy texture, perfect if you want something filling."
- D: "The [Insert Comfort Food Dish] is great for anyone who wants a hearty meal with creamy ingredients."

**Correct Answer:** B
**Tip:** Highlight indulgence and richness, using words like “luxurious” and “comfort” to make it sound more inviting.
</Examples>"""

In [60]:
system_message_content = f"""
<instructions>
You are a helpful assistant which develops comprehensive, scenario-based quiz modules designed to effectively train restaurant staff by enhancing their proficiency in dish appeal.
The quizzes should be engaging, contextually relevant, and varied to ensure high levels of interest without redundancy.
</instructions>

<expected inputs>
Use the following inputs to generate the quiz:
### 1. **Section_Title**: The title of the section the quiz is part of.
### 2. **Title of Quiz**: The name of the quiz that you are generating.
### 3. **Description of Quiz**: A high level description of the quiz that you are generating.
### 4. **Restaurant Name**: The name of the restaurant.
### 5. **Restaurant Location**: The restaurant's location
### 6. **Nationalities**: The top  nationalities of the restaurant's clientele.
### 7. **Cuisine of the Restaurant**: The main cuisine featured at the restaurant.
### 8. **Enhanced Menus**: Enhanced menus which detail the pricing, ingredients, pairing suggestions, cultural narratives, and menu category for each menu item. 
</expected inputs>

<output_requirements>
### 1. **Generate Questions**:
Develop a diverse set of scenario-based questions tailored to the input data. 
Ensure questions are engaging, diverse, framed in real-world scenarios, and not repetitive to maintain quiz freshness and intrigue. 
### 2. **Structure of the Quiz**:
Each quiz should contain 10 questions with multiple-choice answers (A to D), with one correct answer.
</output_requirements>

Use the following examples format, writing style and coherence. Must use each of these styles at least once in your 10 questions.
YOU MUST RESPECT THIS FORMAT AND ALL QUESTIONS MUST HAVE 4 SUGGESTED ANSWERS.
{examples}
"""

In [61]:
from utils import load_json
menu = load_json("../inputs/menus/basic_menu.json")
menu

[{'name': "Pan | Olivade 'casero' comida",
  'appeal': 'A simple yet irresistible homemade olive spread paired with freshly baked bread, perfect to start your meal.',
  'category': 'Starters',
  'ingredients': ['Bread', 'Homemade olive spread'],
  'price': 2.5},
 {'name': 'Caviar Oscietra',
  'appeal': 'Indulge in the luxurious and silky texture of sustainably sourced Oscietra caviar, an unforgettable experience for the palate.',
  'category': 'Starters',
  'ingredients': ['Oscietra caviar'],
  'price': 155.0},
 {'name': 'Ostras Amelie NÂº2',
  'appeal': 'Savor the briny, fresh taste of Amelie nÂº2 oysters, a perfect seafood delicacy that will transport you to the ocean.',
  'category': 'Starters',
  'ingredients': ['Oysters'],
  'price': 5.5},
 {'name': 'Sashimi PORTIXOL',
  'appeal': 'Delicate slices of fresh fish that melt in your mouth, served with a touch of Portixol elegance.',
  'category': 'Starters',
  'ingredients': ['Fresh fish (assorted sashimi)'],
  'price': 27.0},
 {'name

In [62]:
from inputs.restaurant_info.restaurant_info import RESTAURANT_INFO
from inputs.frameworks import content_framework


description= """how to describe dishes in a way that excites guests. This quiz focuses on using vivid, sensory-rich language to transform a simple dish into a must-try menu item.
You’ll practice describing taste, texture, ingredients, and preparation in a way that makes dishes irresistible"""

user_message_content = f"""
### 1. **Title of Section**: Dish Appeal
### 2. **Title of Quiz**: Dish Descriptions
### 3. **Description of Quiz**: {description}
### 4. **Restaurant Name**: {RESTAURANT_INFO.name}
### 5. **Restaurant Location**: {RESTAURANT_INFO.location}
### 6. **Nationalities**: {RESTAURANT_INFO.nationalities}
### 7. **Cuisine of the Restaurant**: {RESTAURANT_INFO.cuisine}
### 8. **menu**: {menu}
    		"""

In [63]:
messages = [{"role": "system", "content": system_message_content},
            {"role": "user", "content": user_message_content}]

In [64]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

openai_model = OpenAI()
completion = openai_model.chat.completions.create(
    model="gpt-4o",
    messages=messages
    )
print("completion end")
response = completion.choices[0].message.content

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


completion end


In [65]:
print(response)

**Title of Section**: Dish Appeal  
**Title of Quiz**: Dish Descriptions  
**Description of Quiz**: How to describe dishes in a way that excites guests. This quiz focuses on using vivid, sensory-rich language to transform a simple dish into a must-try menu item. You'll practice describing taste, texture, ingredients, and preparation in a way that makes dishes irresistible.  
  
### Quiz Questions

**Question 1:** When presenting the "Pan | Olivade 'casero' comida" to guests, which description best highlights its appeal as a starter?  
**Answers:**  
- A: "It's bread with an olive spread, good to begin your meal."  
- B: "Pan | Olivade 'casero' comida features a homemade olive spread paired with freshly baked bread, a simple yet irresistible start to your meal."  
- C: "This dish includes bread and olive spread, great before any meal."  
- D: "The olive spread is homemade and served with fresh bread, nice as a starter."  
**Correct Answer:** B  

**Question 2:** A customer inquires abou

In [66]:
experiment_log = {
    "section": "Dish Appeal",
    "examples": examples,
    "base_menu": menu,
    "menu_enhacement": None,
    "inputs": messages,
    "output": response
}

In [67]:
import json
with open('output_2.json', 'w', encoding='utf-8') as f:
    json.dump(experiment_log, f, ensure_ascii=False, indent=4)